0. Import packages

In [4]:
import pandas as pd
import numpy as np
import random
#import binascii

1. Upload Dataset

In [5]:
from google.colab import files 
uploaded = files.upload() 

Saving train.csv.zip to train.csv.zip


In [6]:
%%shell
unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


2. Randomly pick 10 pairs of duplicate questions, then list up the 20 questions as a question corpus (N=20).

In [23]:
#|--------------------------|
#| Set Number of questions  |
#|__________________________|
size_of_pairs = 10

In [24]:
train_df = pd.read_csv("train.csv",index_col=0)
train_df.head(n=size_of_pairs)

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0


In [25]:
df_shuffled = train_df.sample(frac=1).reset_index(drop=True) # Random shuffling
df1 = df_shuffled["question1"].head(n=size_of_pairs)
df2 = df_shuffled["question2"].head(n=size_of_pairs)
df_shuffled.head(n=size_of_pairs)

,qid1,qid2,question1,question2,is_duplicate
0,119089,119090,Are there border checks between states? Why or...,"If the U.S.-Canada border is like this, why do...",0
1,14799,291557,What do I need to learn to get a fair chance o...,I want to be selected for GSoC. What are some ...,1
2,172437,59166,How do I become better at controlling emotions?,How can I control my emotions?,1
3,198521,198522,What is the inspirational thing that you want ...,What is that one crazy thing that you want to ...,0
4,75216,75217,Can I write a long question on Quora?,Why can't I type a long questions at Quora?,0
5,118107,118108,What Chinese technology is used in America?,How many years does Chinese space technology f...,0
6,391802,391803,What is the best way to connect with god?,Can we communicate with God?,0
7,517056,517057,For a hybrid car competing in a race for the h...,Why does it make sense to become involved in n...,0
8,313010,6974,What is the best civilization in Age of Empire...,What is the best civilization in Age of Empire...,0
9,22732,39113,How do you feel when you fall in love?,How does it feels to be in love?,1


In [26]:
corpus = pd.concat([df1,df2], ignore_index=True) # Make question corpus
print(corpus)

0     Are there border checks between states? Why or...
1     What do I need to learn to get a fair chance o...
2       How do I become better at controlling emotions?
3     What is the inspirational thing that you want ...
4                 Can I write a long question on Quora?
5           What Chinese technology is used in America?
6             What is the best way to connect with god?
7     For a hybrid car competing in a race for the h...
8     What is the best civilization in Age of Empire...
9                How do you feel when you fall in love?
10    If the U.S.-Canada border is like this, why do...
11    I want to be selected for GSoC. What are some ...
12                       How can I control my emotions?
13    What is that one crazy thing that you want to ...
14          Why can't I type a long questions at Quora?
15    How many years does Chinese space technology f...
16                         Can we communicate with God?
17    Why does it make sense to become involved 

3. Make k-shingles and document vectors

In [27]:
k = 3 # length of string
shingles = set() # set of shingles
for sentence in corpus:  
  if sentence[-1] == '\n':
      sentence = sentence[0:-1]      
  words = sentence.split(" ")  
  for index in range(0, len(words)-k+1):
    shingle = words[index]
    for number in range(1, k): # Make k-shingle
      shingle += " " + words[index + number]      
    #shingle = binascii.crc32(shingle) & 0xffffffff
    shingles.add(shingle) # add to set of shingle

In [28]:
print(shingles)

{'getting selected for', 'Can we communicate', 'does it feels', 'want to do', 'for getting into', 'a race for', 'the best way', 'of Empires III?', 'behind the American', 'the highest fuel', 'fall in love?', 'How do you', 'cross it illegally?', 'get a fair', 'do before you', 'tyre and chassis', "why don't more", 'Why or why', 'technology fall behind', 'fair chance of', 'of getting selected', 'border is like', 'the American space', 'highest fuel efficiency', 'be selected for', 'in network marketing?', 'a hybrid car', 'best tyre and', 'to do before', 'become involved in', 'type a long', 'feels to be', 'in Age of', 'connect with god?', 'you want to', 'to connect with', "don't more people", 'need to learn', 'selected for GSoC.', 'selected for GSoC?', 'What are some', 'suggestions for getting', 'time what would', 'one crazy thing', 'are some suggestions', 'sense to become', 'to be in', 'I want to', 'I control my', 'people cross it', 'fall behind the', 'long question on', 'what would be', 'or

In [29]:
print(len(shingles))

166


In [30]:
shingle_list = list(shingles)
document_vector=[]
for num in range(len(corpus)):
  temp = []
  for i in range(len(shingle_list)):
    if shingle_list[i] in corpus[num]:
      temp.append(1)
    else:
      temp.append(0)
  document_vector.append(list(temp))

In [31]:
print(document_vector)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],

4. Make signature matrix M

In [32]:
# hash function : (ax + b) % len
def make_hash(number):
  a = [] # container for a
  b = [] # container for b
  for i in range(number):
    a.append(random.randint(0,1000))
    b.append(random.randint(0,1000))
  lst = []
  lst.append(a)
  lst.append(b)
  return lst

In [45]:
#|------------------------------|
#| Set Number of hashfunctions  |
#|______________________________|
number_of_hashfunctions = 100
lst = make_hash(number_of_hashfunctions)
print(lst)

[[485, 444, 65, 153, 773, 872, 969, 841, 955, 826, 865, 627, 103, 630, 813, 661, 173, 707, 434, 235, 373, 708, 717, 963, 854, 830, 666, 754, 513, 800, 93, 235, 545, 711, 276, 494, 51, 879, 341, 346, 807, 999, 230, 415, 591, 901, 780, 263, 719, 819, 263, 294, 18, 373, 188, 715, 723, 394, 578, 270, 164, 616, 383, 707, 130, 870, 352, 706, 92, 703, 755, 927, 181, 618, 119, 11, 279, 943, 468, 435, 878, 693, 350, 795, 694, 366, 293, 336, 656, 456, 137, 56, 528, 334, 18, 52, 734, 51, 568, 488], [936, 710, 135, 460, 400, 373, 830, 395, 892, 636, 811, 143, 855, 700, 606, 707, 3, 884, 813, 844, 850, 161, 578, 280, 93, 488, 398, 538, 457, 528, 493, 280, 998, 552, 893, 721, 749, 804, 508, 154, 164, 575, 245, 988, 227, 182, 530, 27, 417, 26, 879, 838, 268, 80, 442, 810, 214, 193, 569, 389, 638, 230, 116, 888, 549, 519, 689, 331, 399, 211, 657, 346, 212, 785, 654, 741, 352, 896, 112, 552, 201, 945, 20, 329, 564, 293, 265, 782, 314, 690, 908, 902, 984, 677, 148, 212, 820, 20, 12, 12]]


In [46]:
signature = []

In [47]:
for iteration in range(number_of_hashfunctions):
  temp = []
  for i in range(len(document_vector)):
    first_occur = len(shingles)    
    for j in range(len(shingles)):
      if document_vector[i][j] == 1:
        first_occur = min(first_occur, (lst[0][iteration] * j + lst[1][iteration]) % len(shingles) )
    temp.append(first_occur)    
  signature.append(temp)

In [48]:
print(signature)

[[11, 4, 7, 5, 53, 24, 9, 41, 20, 46, 1, 8, 14, 5, 0, 2, 39, 13, 15, 6], [6, 8, 64, 2, 94, 12, 8, 18, 8, 4, 12, 0, 10, 28, 4, 2, 52, 22, 0, 32], [37, 3, 10, 11, 5, 47, 27, 4, 24, 8, 0, 7, 40, 12, 1, 22, 34, 23, 24, 70], [0, 4, 16, 12, 75, 3, 5, 6, 42, 9, 19, 15, 2, 12, 14, 1, 61, 17, 37, 28], [9, 3, 4, 21, 16, 0, 13, 23, 7, 22, 8, 6, 72, 21, 2, 25, 11, 5, 1, 102], [15, 9, 7, 1, 25, 25, 17, 1, 27, 5, 9, 3, 19, 1, 11, 29, 83, 35, 3, 15], [7, 0, 10, 23, 30, 34, 4, 2, 9, 14, 5, 16, 26, 1, 20, 12, 3, 47, 9, 27], [3, 10, 20, 8, 24, 1, 66, 0, 4, 7, 2, 5, 77, 8, 19, 26, 74, 6, 4, 52], [5, 3, 11, 17, 106, 16, 15, 0, 20, 26, 30, 9, 7, 17, 32, 4, 21, 8, 50, 1], [14, 8, 18, 0, 100, 8, 6, 2, 10, 12, 4, 42, 48, 0, 16, 0, 28, 4, 10, 14], [10, 13, 56, 9, 38, 23, 21, 3, 21, 2, 28, 6, 1, 9, 7, 42, 14, 0, 21, 45], [0, 28, 3, 32, 41, 61, 10, 1, 19, 42, 21, 7, 27, 20, 22, 2, 18, 5, 35, 14], [6, 18, 44, 2, 22, 11, 38, 0, 1, 28, 9, 7, 66, 2, 16, 19, 32, 12, 1, 64], [46, 8, 12, 22, 0, 10, 8, 24, 20, 6, 4, 4, 

5. Evaluate Min-Hashing<br>
(Approximated similarity vs Gold similarity)

In [49]:
print(signature)
print(len(signature))
print(document_vector)
print(len(document_vector))

[[11, 4, 7, 5, 53, 24, 9, 41, 20, 46, 1, 8, 14, 5, 0, 2, 39, 13, 15, 6], [6, 8, 64, 2, 94, 12, 8, 18, 8, 4, 12, 0, 10, 28, 4, 2, 52, 22, 0, 32], [37, 3, 10, 11, 5, 47, 27, 4, 24, 8, 0, 7, 40, 12, 1, 22, 34, 23, 24, 70], [0, 4, 16, 12, 75, 3, 5, 6, 42, 9, 19, 15, 2, 12, 14, 1, 61, 17, 37, 28], [9, 3, 4, 21, 16, 0, 13, 23, 7, 22, 8, 6, 72, 21, 2, 25, 11, 5, 1, 102], [15, 9, 7, 1, 25, 25, 17, 1, 27, 5, 9, 3, 19, 1, 11, 29, 83, 35, 3, 15], [7, 0, 10, 23, 30, 34, 4, 2, 9, 14, 5, 16, 26, 1, 20, 12, 3, 47, 9, 27], [3, 10, 20, 8, 24, 1, 66, 0, 4, 7, 2, 5, 77, 8, 19, 26, 74, 6, 4, 52], [5, 3, 11, 17, 106, 16, 15, 0, 20, 26, 30, 9, 7, 17, 32, 4, 21, 8, 50, 1], [14, 8, 18, 0, 100, 8, 6, 2, 10, 12, 4, 42, 48, 0, 16, 0, 28, 4, 10, 14], [10, 13, 56, 9, 38, 23, 21, 3, 21, 2, 28, 6, 1, 9, 7, 42, 14, 0, 21, 45], [0, 28, 3, 32, 41, 61, 10, 1, 19, 42, 21, 7, 27, 20, 22, 2, 18, 5, 35, 14], [6, 18, 44, 2, 22, 11, 38, 0, 1, 28, 9, 7, 66, 2, 16, 19, 32, 12, 1, 64], [46, 8, 12, 22, 0, 10, 8, 24, 20, 6, 4, 4, 

In [50]:
# approximated sim -> signature : signature
# gold sim -> shingle : document_vector
a_sim = [0,]*size_of_pairs
g_sim = [0,]*size_of_pairs
gap = [0,]*size_of_pairs
# calculate approximated similarity
for i in range(number_of_hashfunctions):
  for j in range(size_of_pairs):
    if signature[i][j] == signature[i][j+size_of_pairs]:
      a_sim[j] += 1
for i in range(len(a_sim)):
  a_sim[i] /= len(signature)
print(a_sim)

# calculate gold similarity
for i in range(size_of_pairs):
  for j in range(len(shingles)):
    if document_vector[i][j] == document_vector[i+size_of_pairs][j]:
      g_sim[i] += 1
for i in range(len(g_sim)):
  g_sim[i] /= len(shingles)
print(g_sim)

# calculate gap : abs(simA-simB)
for i in range(size_of_pairs):
  gap[i] = abs(a_sim[i] - g_sim[i])
print(gap)

# calculate average gap
avg = 0.0
for i in range(size_of_pairs):
  avg += gap[i]
avg /= size_of_pairs
print(avg)

[0.02, 0.03, 0.06, 0.48, 0.11, 0.02, 0.02, 0.03, 0.74, 0.02]
[0.8795180722891566, 0.8373493975903614, 0.9397590361445783, 0.9457831325301205, 0.927710843373494, 0.9036144578313253, 0.9397590361445783, 0.8012048192771084, 0.9819277108433735, 0.9216867469879518]
[0.8595180722891566, 0.8073493975903614, 0.8797590361445784, 0.4657831325301205, 0.817710843373494, 0.8836144578313253, 0.9197590361445783, 0.7712048192771084, 0.2419277108433735, 0.9016867469879518]
0.7548313253012047
